<a href="https://colab.research.google.com/github/mafudsetiono/A-B-Testing-Campaign/blob/main/A_B_Testing_Campaign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objective
Tujuan utama dari pengujian A/B ini adalah untuk menentukan apakah strategi iklan berbayar (grup 'ad') menghasilkan tingkat konversi yang secara statistik lebih tinggi dibandingkan dengan strategi iklan layanan masyarakat (grup 'psa').

Metrik utama yang akan diukur untuk mencapai tujuan ini adalah Tingkat Konversi (Conversion Rate).

In [ ]:
#Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from statsmodels.stats.proportion import proportions_ztest
warnings.filterwarnings('ignore')

#Read Data

In [ ]:
df = pd.read_csv('marketing_AB.csv')

In [ ]:
#Hapus Unnamed
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 23.0+ MB


In [ ]:
# Kelompokkan data berdasarkan 'test group' dan hitung total pengguna dan total konversi
summary = df.groupby('test group')['converted'].agg(['count', 'sum']).reset_index()
summary.columns = ['test group', 'total_users', 'total_conversions']

# Hitung Tingkat Konversi
summary['conversion_rate'] = summary['total_conversions'] / summary['total_users']

# Tampilkan hasil deskriptif
print("--- Hasil Analisis Data Deskriptif (Tingkat Konversi) ---")
summary_display = summary.copy()
summary_display['conversion_rate'] = summary_display['conversion_rate'].map('{:.4f}%'.format)
summary_display['total_users'] = summary_display['total_users'].map('{:,}'.format)
summary_display['total_conversions'] = summary_display['total_conversions'].map('{:,}'.format)
print(summary_display)

--- Hasil Analisis Data Deskriptif (Tingkat Konversi) ---
  test group total_users total_conversions conversion_rate
0         ad     564,577            14,423         0.0255%
1        psa      23,524               420         0.0179%


In [ ]:
summary.head()

,test group,total_users,total_conversions,conversion_rate
0,ad,564577,14423,0.025547
1,psa,23524,420,0.017854


In [ ]:
df.value_counts('test group')

,count
test group,
ad,564577
psa,23524


# Uji Hipotesis
(H0) Hipotesis Nol menyatakan bahwa strategi iklan berbayar tidak memberikan peningkatan signifikan terhadap Tingkat Konversi dibandingkan dengan strategi iklan layanan masyarakat.

(H1) Hipotesis Alternatif menyatakan bahwa strategi iklan berbayar secara signifikan lebih efektif dalam meningkatkan Tingkat Konversi.

In [ ]:
# Group
group_a = df[ df['test group'] == 'psa']
group_b = df[ df['test group'] == 'ad']

# Order
conversion = [ group_b['converted'].sum(), group_a['converted'].sum() ]
sample = [ group_b['converted'].shape[0], group_a['converted'].shape[0] ]

print(conversion)
print(sample)

[np.int64(14423), np.int64(420)]
[564577, 23524]


In [ ]:
z_stat, p_value = proportions_ztest(conversion, sample, alternative='larger')

print(f"P-value: {p_value:.6f}")
print(f"CR control: {(group_a['converted'].sum()/group_a['converted'].shape[0]):.4f}")
print(f"CR variant: {(group_b['converted'].sum()/group_b['converted'].shape[0]):.4f}")

P-value: 0.000000
CR control: 0.0179
CR variant: 0.0255


Karena P-value (0.000000) jauh lebih kecil dari Level

Signifikansi 0.05, kita menolak Hipotesis Nol H0.
Strategi iklan berbayar (ad) menghasilkan Tingkat Konversi yang secara statistik signifikan dan lebih tinggi daripada strategi iklan layanan masyarakat (psa).

Rekomendasi Bisnis: Bisnis harus meluncurkan dan mengoptimalkan strategi iklan berbayar karena telah terbukti lebih efektif.

# Rata-rata jumlah iklan dilihat

In [ ]:
print(group_a['total ads'].mean())
print(group_b['total ads'].mean())

24.761137561639178
24.823365103431417
